# C2Cシェアサイクルシステム

## 問題設定
個人間で自転車をシェアリングし，可能な限り乗り捨て可能とするシステムについて考える．利用可能な自転車は複数台存在し，それぞれが任意のポイントに駐輪されているとする．本システムのユーザーのサービス利用リクエストを時間幅Tの間隔で分割し，それぞれのリクエストに合わせた最適な自転車を割り当てる．

サービス利用後に任意の場所に駐輪された自転車を定位置に再配置するコストを最小化し，　より多くのユーザーに自転車を割り当てることを目的として，最適化な自転車をユーザーに割り当てる．

なお，ここではMVPとして，時間幅T=1(分)とした小規模のデータを利用することとし，最終アウトプットとしては「どのユーザーにどの自転車が割り当てられたか」と「移動後の自転車の配置状況・利用可能状況」を期待する．

[fig]

### データ
データとしては，以下のような情報を必要とする．
- 定数
 - $T$：ユーザーリクエストを区切る時間幅
- 変数
 - $J$：ユーザーリクエストデータ(データフレーム？)
 - $B$：利用可能な自転車データ(データフレーム？)

 etc

In [ ]:
# ライブラリのインストール
!pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 38.7 MB/s eta 0:00:00


In [ ]:
import pulp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# データの準備

'''locationID検索用CSV'''
df_locations = pd.read_csv('/content/taxi_zone_lookup_with_coordinates.csv')
# df_locations.set_index("LocationID", inplace=True)
print(df_locations.head())


# LocationIDから経度と緯度をタプルで返す関数
def get_coordinates_by_location_id(location_id):
    row = df_locations[df_locations['LocationID'] == location_id]
    if not row.empty:
        return (row.iloc[0]['Latitude'], row.iloc[0]['Longitude'])
    else:
        return None

   LocationID        Borough                     Zone service_zone   Latitude  \
0           1            EWR           Newark Airport          EWR  40.689531   
1           2         Queens              Jamaica Bay    Boro Zone  40.603994   
2           3          Bronx  Allerton/Pelham Gardens    Boro Zone  40.865229   
3           4      Manhattan            Alphabet City  Yellow Zone  40.725102   
4           5  Staten Island            Arden Heights    Boro Zone  40.563700   

   Longitude  
0 -74.174462  
1 -73.835412  
2 -73.842739  
3 -73.979583  
4 -74.191603  


In [ ]:
'''自転車の集合'''
# ランダムシードを設定して、ランダムに10個選択
np.random.seed(42)
random_sample = df_locations.sample(n=10)

# Bike IDを設定
random_sample['Bike ID'] = range(10)

# 緯度と経度をホームポジションとカレントポジションに設定
random_sample['Home Position'] = list(zip(random_sample['Latitude'], random_sample['Longitude']))
random_sample['Current Location'] = random_sample['Home Position']

# 結果のデータフレームを整形
B = random_sample[['Bike ID', 'Home Position', 'Current Location']]
B.set_index("Bike ID", inplace=True)

# データの中身を確認
B

,Home Position,Current Location
Bike ID,,
0,"(40.67677, -73.8437461)","(40.67677, -73.8437461)"
1,"(40.8241451, -73.9500618)","(40.8241451, -73.9500618)"
2,"(40.6907711, -73.9766245)","(40.6907711, -73.9766245)"
3,"(40.68562615, -73.98417065807277)","(40.68562615, -73.98417065807277)"
4,"(40.67592055, -73.78496487588887)","(40.67592055, -73.78496487588887)"
5,"(40.76883397436158, -73.95193997045698)","(40.76883397436158, -73.95193997045698)"
6,"(40.70533183168504, -73.95019177498656)","(40.70533183168504, -73.95019177498656)"
7,"(40.8473226, -73.7865218)","(40.8473226, -73.7865218)"
8,"(40.750201, -73.993104)","(40.750201, -73.993104)"


In [ ]:
'''ユーザーリクエストの集合'''
# ParquetファイルのURL
url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet'

# Parquetファイルを読み込む
df = pd.read_parquet(url)

# 指定されたカラムのみを含むデータフレームを取得
df_requests = df[['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'PULocationID', 'DOLocationID']]

# データのフィルタリング
# 2023年1月1日以前のデータを削除
df_requests = df_requests[df_requests['tpep_pickup_datetime'] >= '2023-01-01']
# 2023年2月1日以降のデータを削除
df_requests = df_requests[df_requests['tpep_pickup_datetime'] <= '2023-01-31']

# ピックアップタイムの昇順で並び替え
df_requests = df_requests.sort_values(by='tpep_pickup_datetime')

# インデックスをリセット
df_requests = df_requests.reset_index(drop=True)

# フィルタリングされたデータの先頭を表示
print(df_requests.head())

# データフレームの情報を表示
print(df_requests.info())

  tpep_pickup_datetime tpep_dropoff_datetime  PULocationID  DOLocationID
0  2023-01-01 00:00:00   2023-01-01 00:08:00            42            41
1  2023-01-01 00:00:05   2023-01-01 00:26:27           249           186
2  2023-01-01 00:00:06   2023-01-01 00:05:44           125            68
3  2023-01-01 00:00:08   2023-01-01 00:11:24            42           244
4  2023-01-01 00:00:09   2023-01-01 00:15:10            79           231
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2966346 entries, 0 to 2966345
Data columns (total 4 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   tpep_pickup_datetime   datetime64[us]
 1   tpep_dropoff_datetime  datetime64[us]
 2   PULocationID           int64         
 3   DOLocationID           int64         
dtypes: datetime64[us](2), int64(2)
memory usage: 90.5 MB
None


In [ ]:
# モデリングするためにユーザーリクエストデータを整形する

# tpep_pickup_datetimeをdatetime型に変換
df_requests['tpep_pickup_datetime'] = pd.to_datetime(df_requests['tpep_pickup_datetime'])
df_requests['tpep_dropoff_datetime'] = pd.to_datetime(df_requests['tpep_dropoff_datetime'])

# 一か月分のデータを一分ごとに分割
start_time = df_requests['tpep_pickup_datetime'].min()
end_time = df_requests['tpep_pickup_datetime'].max()

# 最初の1分のデータを抽出
start_time = df_requests['tpep_pickup_datetime'].min()
end_time = start_time + pd.Timedelta(minutes=1)

# まずは最初の1分間のリクエストをユーザーリクエストの集合として扱う
J = df_requests[(df_requests['tpep_pickup_datetime'] >= start_time) & (df_requests['tpep_pickup_datetime'] < end_time)]

# type(J)
J

,tpep_pickup_datetime,tpep_dropoff_datetime,PULocationID,DOLocationID
0,2023-01-01 00:00:00,2023-01-01 00:08:00,42,41
1,2023-01-01 00:00:05,2023-01-01 00:26:27,249,186
2,2023-01-01 00:00:06,2023-01-01 00:05:44,125,68
3,2023-01-01 00:00:08,2023-01-01 00:11:24,42,244
4,2023-01-01 00:00:09,2023-01-01 00:15:10,79,231
5,2023-01-01 00:00:13,2023-01-01 00:12:52,132,7
6,2023-01-01 00:00:18,2023-01-01 00:09:34,238,262
7,2023-01-01 00:00:22,2023-01-01 00:26:23,136,233
8,2023-01-01 00:00:35,2023-01-01 00:25:12,132,17
9,2023-01-01 00:00:47,2023-01-01 00:04:32,79,107


In [ ]:
import numpy as np
from pandas import DataFrame
from geopy.distance import geodesic


def generate_after_trip_distances(
    df_bikes: DataFrame,
    df_requests: DataFrame,
) -> np.ndarray:

    # 自転車とリクエストの数
    num_bikes = len(df_bikes)
    num_requests = len(df_requests)

    # 移動後の距離行列 d を作成 (d[b, j] が利用者 j が移動した後の自転車 b とその定位置までの距離)
    # 距離行列を初期化
    after_trip_distances = np.zeros((num_bikes, num_requests))
    for b in range(num_bikes):
        home_position = df_bikes.loc[b, 'Home Position']
        for j in range(num_requests):
            request_destination_id = df_requests.loc[j, 'DOLocationID']
            request_destination = get_coordinates_by_location_id(request_destination_id)
            after_trip_distances[b, j] = geodesic(
                home_position, request_destination
            ).m  # 単位はメートル

    return after_trip_distances

In [ ]:
# デバッグ
result = generate_after_trip_distances(B, J)
print(result)

[[16324.6965866  15025.69004275 15406.86232578 19990.18350913
  14907.19068104 12856.90047253 14196.22025405 13245.07257533
  10638.86535405 13535.28056899 16373.93306076 11340.10038402]
 [ 2880.37709413  8979.25090277  9655.74897413  2107.14841695
  12971.40478868  6025.30017515  5006.95320151  8415.52519669
  11719.56250095 10252.2746341   2851.99880036 14979.9612261 ]
 [12110.43982081  6744.87746968  6527.93089092 17013.87720116
   4186.64777432  9835.22134701 10204.78938704  6572.77049439
   3524.74055425  5008.93846511 12134.94413876     0.        ]
 [12791.84800114  7210.55299741  6909.50261733 17700.2961098
   4220.59321199 10619.22053993 10936.19640361  7234.50307073
   4339.58949175  5540.51132054 12814.43064257   856.33837516]
 [19620.20207123 19425.83248026 19896.42120853 22539.76440839
  19723.18936374 16265.6081012  17680.1909753  17488.56913273
  15290.03178085 18081.58266884 19671.64681594 16285.95097795]
 [ 3269.84196753  4045.16769894  4861.29000959  8122.40191257
   7



---



In [ ]:
num_people = 30
num_cars = 10
car_capacity = 6

In [ ]:
import numpy as np
from geopy.distance import geodesic


def generate_problem(
    num_people: int,
    num_cars: int,
    latitude_range: tuple[float, float],
    longitude_range: tuple[float, float],
    seed=999,
) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    rng = np.random.default_rng(seed)

    # 利用者と車の座標を生成
    people_coords = rng.uniform(
        low=np.array([latitude_range[0], longitude_range[0]]),
        high=np.array([latitude_range[1], longitude_range[1]]),
        size=(num_people, 2),
    )
    car_coords = rng.uniform(
        low=np.array([latitude_range[0], longitude_range[0]]),
        high=np.array([latitude_range[1], longitude_range[1]]),
        size=(num_cars, 2),
    )

    # 距離行列 d を作成 (d[i, k] が利用者 i と車 k の距離)
    distances = np.zeros((num_people, num_cars))
    for i in range(num_people):
        for k in range(num_cars):
            distances[i, k] = geodesic(
                people_coords[i], car_coords[k]
            ).m  # 単位はメートル

    return people_coords, car_coords, distances

In [ ]:
# 船橋駅周辺
latitude_range = (35.675500, 35.76)
longitude_range = (139.9, 140.08)

people_coords, car_coords, distances = generate_problem(
    num_people, num_cars, latitude_range, longitude_range
)

print(people_coords)

[[ 35.74131071 139.93100475]
 [ 35.7357969  140.03506031]
 [ 35.68995872 140.06234642]
 [ 35.68748054 140.01002117]
 [ 35.75247469 139.93433919]
 [ 35.72260029 140.01017146]
 [ 35.71884253 140.01489185]
 [ 35.71626861 139.95198507]
 [ 35.67898904 139.93366338]
 [ 35.70025301 140.03115863]
 [ 35.71729483 139.92856181]
 [ 35.71207214 139.97298586]
 [ 35.71300278 140.02087558]
 [ 35.70071972 139.96650166]
 [ 35.72328468 139.9091992 ]
 [ 35.70980571 140.03539191]
 [ 35.75526812 139.92465517]
 [ 35.72392911 140.01290959]
 [ 35.73556492 140.07232102]
 [ 35.70178617 139.93077819]
 [ 35.72819221 140.0526187 ]
 [ 35.7572412  139.99425738]
 [ 35.68369004 140.07177064]
 [ 35.74448693 140.0008116 ]
 [ 35.67820935 140.03679134]
 [ 35.69225639 140.02936744]
 [ 35.72890758 140.05501428]
 [ 35.71019966 139.98791099]
 [ 35.7269548  140.06041128]
 [ 35.74735725 139.93178367]]


In [ ]:
import folium


def plot_people_and_cars(
    people_coords: np.ndarray,
    car_coords: np.ndarray,
    latitude_range: tuple[float, float],  # 描画範囲 (緯度)
    longitude_range: tuple[float, float],  # 描画範囲 (経度)
):
    m = folium.Map(
        [sum(latitude_range) / 2, sum(longitude_range) / 2],
        tiles="OpenStreetMap",
        zoom_start=12,
    )

    for latitude, longitude in people_coords:
        folium.Marker(
            location=(latitude, longitude),
            icon=folium.Icon(icon="user", prefix="fa", color="orange"),
        ).add_to(m)

    for latitude, longitude in car_coords:
        folium.Marker(
            location=(latitude, longitude),
            icon=folium.Icon(icon="car", prefix="fa", color="green"),
        ).add_to(m)

    return m

In [ ]:
plot_people_and_cars(people_coords, car_coords, latitude_range, longitude_range)

In [ ]:
import numpy as np

average = distances.mean()
print(average)
std = distances.std()
print(std)
distances: np.ndarray = (distances - average) / std
print(np.array(distances))

7320.4042889845805
3665.1203561497346
[[-1.61654573e+00  1.30134364e+00 -4.87700271e-02  1.55047539e+00
  -4.97545872e-01 -9.08789545e-01 -2.60408815e-01  2.40721179e-01
   1.49190870e+00 -7.27968794e-02]
 [ 2.23711291e-01 -1.18101945e+00  1.50277992e+00 -8.51196511e-01
  -2.89245146e-01 -4.54569363e-01  1.35949652e+00 -2.15599929e-01
  -8.21922697e-01  1.38011935e+00]
 [ 1.21805135e+00 -2.31777916e-01  1.73959464e+00 -1.34695654e+00
   2.16438234e-01  5.23676320e-01  1.67412866e+00 -2.94269400e-01
  -1.50625990e+00  1.59514674e+00]
 [ 1.80369790e-01  2.80143029e-01  4.46112957e-01 -3.66054680e-01
  -9.48935500e-01 -4.13326217e-01  3.80225751e-01 -1.58782043e+00
  -5.16461404e-01  3.02061677e-01]
 [-1.43668200e+00  1.21466111e+00  2.99059379e-01  1.57638066e+00
  -2.98966177e-01 -8.29758455e-01  8.74218941e-02  4.46535601e-01
   1.53406425e+00  2.74417634e-01]
 [-3.38928751e-01 -4.49965748e-01  7.71765609e-01 -4.83364477e-01
  -1.02053374e+00 -1.05003774e+00  6.32130726e-01 -8.63331123

In [ ]:
!pip install amplify

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 25.0 MB/s eta 0:00:00


In [ ]:
from amplify import VariableGenerator

gen = VariableGenerator()
q = gen.array("Binary", num_people, num_cars)

In [ ]:
from amplify import sum

alpha = 1.0

# 第一項: 利用者の移動距離を短くする
distance_objective = (distances * q).sum()

# 第二項: 乗車率の二乗和を大きくする
occupancies = q.sum(axis=0) / car_capacity  # 乗車率
occupancy_objective = (occupancies**2).sum()

objective = distance_objective - alpha * occupancy_objective

In [ ]:
from amplify import one_hot

one_person_one_car_constraints = one_hot(q, axis=1)

In [ ]:
from amplify import less_equal

car_capacity_constraints = less_equal(q, car_capacity, axis=0)

In [ ]:
one_person_one_car_constraints *= np.max(distances) + alpha * 1
car_capacity_constraints *= np.max(distances) + alpha * 2 / car_capacity

In [ ]:
from amplify import Model

model = objective + one_person_one_car_constraints + car_capacity_constraints

In [ ]:
from amplify import FixstarsClient, solve
from datetime import timedelta

client = FixstarsClient()
client.parameters.timeout = timedelta(milliseconds=2000)  # タイムアウトは 2000 ms
# client.token = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"  # ローカル環境等で使用する場合は、Fixstars Amplify AE のアクセストークンを入力してください。

result = solve(model, client)
if len(result) == 0:
    # 実行可能解が見つかっていなければ例外を投げる
    raise RuntimeError("No feasible solution was found.")

RuntimeError: 401: Unauthorized